In [1]:
import numpy as np
import pandas as pd
import time
from collections import namedtuple, deque
from numpy import random as rnd
from functools import reduce
import copy
import matplotlib.pyplot as plt
import sklearn.linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from numpy import random as rnd

%matplotlib inline


In [ ]:
# Programming assignment 3.1.1&2 minimizing the weighted sum of completion times.

def score1(w, l):
    return w - l

def score2(w, l):
    return w/float(l)

def sortJobs (jobList, scoreFunc):
    scoreJobList = [(w,l, scoreFunc (w,l)) for w, l in jobList ]
    scoreJobList.sort(key=lambda tup: tup[0], reverse=True)   # sort by weight first to resolve ties
    scoreJobList.sort(key=lambda tup: tup[2], reverse=True)   # sort by score function
    return scoreJobList;

def loadData(fName):
    with open(fName,'r') as fileObj:
        lines = fileObj.readlines()
        numJobs = int (lines[0].strip())
        jobList = [ (int(line.split()[0]), int(line.split()[1])) for line in lines[1:]]
    return jobList

def sumCompletionTimes (sortedJobList):
    weightedSum = 0
    lengthSum = 0
    
    for job in sortedJobList:
        lengthSum += job[1]
        weightedSum += job[0] * lengthSum
        
    return weightedSum, lengthSum
    
def main(fName):
    %cd /Users/AM/Documents/_Courses/Algorithms/Misc
    jobList = loadData(fName);
    scoreJobList = sortJobs(jobList, score1)    #change to score1 or score2
    weightedSum, lengthSum = sumCompletionTimes (scoreJobList)
    print(weightedSum, lengthSum)
    
if __name__ == '__main__':
    main('text_3_1_jobs.txt')
    
#Score1: 69119377652 510289
#Score2: 67311454237 510289

In [ ]:
# Programming assignment 3.1.3 Prim's Minimum Spanning tree.


def getGraph(fPath = "text_3_1_edges.txt"):
    edgeList = []
    %cd /Users/AM/Documents/_Courses/Algorithms/Misc
    with open(fPath) as f:
        lines = f.read().splitlines()
        numNodes, numEdges = [int(el) for el in lines[0].split() ]
        for line in lines[1:]:
            node1, node2, edgeCost = [int(el) for el in line.split() ]
            edgeList.append((node1, node2, edgeCost))
            
    return edgeList, numNodes, numEdges


def prim(edgeList, numNodes):
    
    X = set()   # explored nodes
    V = set(range(1, numNodes+1))   # unexplored nodes
    E = set()   # set edges of MST
    totalCost = 0   # total sum of edge cost of the MST
    
    start = 1
    X.add(start)
    V.remove(start)
    
    while V:
        # look at all edges crossing the X and V sets
        lowestCost = float('inf')
        foundNodeX = None
        foundNodeV = None
        new_edge_in_X = None
        
        for edge in edgeList:
            if edge[0] in X and edge[1] in V:
                if edge[2] < lowestCost:
                    foundNodeX = edge[0]
                    foundNodeV = edge[1]
                    lowestCost = edge[2]
                    new_edge_in_X = edge
                    
            elif edge[1] in X and edge[0] in V:
                if edge[2] < lowestCost:
                    foundNodeX = edge[1]
                    foundNodeV = edge[0]
                    lowestCost = edge[2]
                    new_edge_in_X = edge

        
        #print("FoundNodeX: {0}, FoundNodeV: {1}".format(foundNodeX, foundNodeV))
        edgeList.remove(new_edge_in_X)
        X.add(foundNodeV)
        V.remove(foundNodeV)
        E.add((foundNodeX, foundNodeV, lowestCost))
        totalCost += lowestCost
        
    return E, totalCost
        
if __name__ == '__main__':
    edgeList, numNodes, numEdges = getGraph()
    t0 = time.time()
    E, totalCost = prim(edgeList, numNodes)
    t1 = time.time()

    print(f"Total cost: {totalCost}, time spent {t1 - t0}")

In [ ]:
# Programming assignment 3.2.1 clustering
class Clust:
    
    def __init__ (self, file_name):
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        with open(file_name) as file:
            lines = file.read().splitlines()
        self.n_nodes = self.n_clust = int(lines[0])
        self.min_dist = 0
        self.edges = [] # [node_1, node_2, cost]
        self.node_to_leader = {} # {node as key: leader}
        self.leader_to_nodes = {} # {leader as key: [nodes]}

        for line in lines[1:]:
            node_1, node_2, cost = [int(a) for a in line.split()]
            self.edges.append((node_1, node_2, cost))
        
        self.edges.sort(key=lambda x: x[2])
        self.edges = deque(self.edges)
        
        for i in range(self.n_nodes + 1):
            self.node_to_leader[i] = i
            self.leader_to_nodes[i] = [i]      

    def merge_nodes(self, n1, n2):
        leader_1 = self.node_to_leader[n1]
        leader_2 = self.node_to_leader[n2]
        for node in self.leader_to_nodes[leader_2]: # no time to implement change of the leader for the smallest cluster
            self.node_to_leader[node] = leader_1
        self.leader_to_nodes[leader_1].extend(self.leader_to_nodes[leader_2])
        self.n_clust -= 1
        del self.leader_to_nodes[leader_2]

        
    def cluster(self):    
        while self.n_clust > 4:
            edge = self.edges.popleft()
            node_1, node_2 = edge[0], edge[1]
            if self.node_to_leader[node_1] != self.node_to_leader[node_2]:
                self.merge_nodes(node_1, node_2)
        
        while self.node_to_leader[node_1] == self.node_to_leader[node_2]:    
            edge = self.edges.popleft()
            node_1, node_2 = edge[0], edge[1]
        self.min_dist = edge[2]

if __name__ == "__main__":
    c = Clust('text_3_2_clust.txt')
    c.cluster()
    print(c.min_dist)
            


In [107]:
# Programming assignment 3.2.2 clustering version 1
class Clust:
    
    def __init__ (self, file_name):
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        with open(file_name) as file:
            lines = file.read().splitlines()
        self.n_nodes, self.n_bits = [int(a) for a in lines[0].split()]
        self.n_clust = self.n_nodes
        self.node_to_leader = {} # {node as key: leader}
        self.leader_to_nodes = {} # {leader as key: [nodes]}

        for line in lines[1:]:
            node_str = line.replace(' ', '')
            node = int(node_str, 2)
            
            if node in self.leader_to_nodes.keys():
                self.n_clust -= 1
            else:    
                self.leader_to_nodes[node] = [node]
                self.node_to_leader[node] = node

    def invert_bit(self, node, bit_number): 
        """inverts #bit_number (starting from the right)"""
        x = node ^ (1<<bit_number)
        return(x)
    
    def find_dist_2(self, node): 
        """# returns the list of all the numbers with Humming distance (bin) <=2"""
        result = []
        for i in range(self.n_bits):
            node_one_inv = self.invert_bit(node, i)
            for j in range(self.n_bits):
                node_two_inv = self.invert_bit(node_one_inv, j)
                if node_two_inv in self.node_to_leader.keys() and node_two_inv != node:    
                    result.append(node_two_inv)
        return(result)
    
    def merge(self, n1, n2):
        leader_1 = self.node_to_leader[n1]
        leader_2 = self.node_to_leader[n2]
        if True: #len(self.leader_to_nodes[leader_1]) > len(self.leader_to_nodes[leader_2]):
            leader_stay, leader_away = leader_1, leader_2
        else: 
            leader_stay, leader_away = leader_2, leader_1
        for node in self.leader_to_nodes[leader_away]: # no time to implement change of the leader for the smallest cluster
            self.node_to_leader[node] = leader_stay
        self.leader_to_nodes[leader_stay].extend(self.leader_to_nodes[leader_away])
        self.n_clust -= 1
        del self.leader_to_nodes[leader_away]
    
    def clustering(self):
        for node in self.node_to_leader.keys():
            print(f'n {self.n_clust:9} \r', end = '')
            next_nodes = self.find_dist_2(node)
            leader = self.node_to_leader[node]
            for candidate in next_nodes:
                if self.node_to_leader[candidate] != leader:
                    self.merge(node, candidate)            
        
if __name__ == "__main__":
    c = Clust('text_3_2_2_clust.txt')
    #c.clustering()
    print('\n')
    print(c.n_clust, c.n_nodes, len(c.leader_to_nodes.keys()))
                    
            


/Users/AM/Documents/_Courses/Algorithms/misc


198788 200000 198788


In [135]:
# Programming assignment 3.3.1 huffman THINK AGAIN
class Huffman:
    
    def __init__ (self, file_name):
        %cd /Users/AM/Documents/_Courses/Algorithms/Misc
        with open(file_name) as file:
            lines = file.read().splitlines()
        self.n_symbols = int(lines[0]) 
        self.weights = []
        for i, line in enumerate(lines[1:]):
            self.weights.append(int(line))
        self.weights.sort()
        self.Node = namedtuple('Node', ['next_left', 'next_right', 'depth', 'weight'])
        self.nodes = [] 

    def test(self):
        a = self.Node(3,2,1,0)
        self.nodes.append(a)
        
    
if __name__ == "__main__":
    h = Huffman('text_3_3_1_huffman.txt')
    print(h.weights[:10])
    print(type(h.weights[9]))
    print(h.n_symbols)
    h.test()
    print(h.nodes[1])

/Users/AM/Documents/_Courses/Algorithms/misc
[1873, 12710, 37164, 40882, 57802, 61282, 64537, 70116, 83429, 90939]
<class 'int'>
1000
1


In [ ]:
# Programming assignment 3.4.1 Knapsack #think again
'''think again'''